In [1]:
#CELL DEDICATED TO IMPORT PACKAGES USEFULL FOR OUR PURPOSES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
from keras.models import Sequential
from keras.layers import Dense
import os
import cv2
from keras.utils import to_categorical
from keras import optimizers
from keras import initializers
from keras.callbacks import EarlyStopping
from keras.callbacks import History
from keras.layers.normalization import BatchNormalization
from keras.applications.vgg16 import VGG16
from keras.utils import plot_model
from sklearn.metrics import confusion_matrix
from keras.models import Model
from keras.preprocessing import image
from keras.layers import Conv2D,Flatten,Dropout, MaxPooling2D,AveragePooling2D

import import_ipynb
import utils


Using TensorFlow backend.


importing Jupyter notebook from utils.ipynb


In [7]:
# SOME IMPORTANT GLOBAL VARIABLE
labels = ["Bedroom","Coast","Forest","HighWay","Industrial","InsideCity","Kitchen","LivingRoom","Mountain","Office","OpenCountry","Store","Street","Suburb","TallBuilding"]
train_dir = '../images/train/'
test_dir = '../images/test/'
from keras.preprocessing import image

# CREATION OF TRAINING DATA

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [9]:
list_of_images = utils.list_of_path(labels,train_dir)
train_data,train_labels = utils.read_and_process_images(list_of_images,dimension=224)


In [10]:
train_labels.shape

(1500,)

In [11]:




from keras.applications.vgg16 import preprocess_input
cont = 1500
train_images =  []
for im in list_of_images:
    temp_img=image.load_img(im,target_size=(224,224))
    temp_img=image.img_to_array(temp_img)
    train_images.append(temp_img)

    

train_images=np.array(train_images)
train_img=preprocess_input(train_images)


train_labels = le.fit_transform(train_labels)
train_labels_dummy=to_categorical(train_labels,15)



In [12]:
train_labels.shape

(1500,)

# CREATION OF TEST DATA

In [13]:
list_of_images = utils.list_of_path(labels,test_dir)
test_data,test_labels = utils.read_and_process_images(list_of_images,dimension=224)




from keras.applications.vgg16 import preprocess_input
test_images =  []

for im in list_of_images:
    temp_img=image.load_img(im,target_size=(224,224))
    temp_img=image.img_to_array(temp_img)
    test_images.append(temp_img)

    
test_images=np.array(test_images)
test_img=preprocess_input(test_images)




test_labels=le.fit_transform(test_labels)
test_labelsdummy=to_categorical(test_labels,15)

In [14]:
def vgg16_model(img_rows, img_cols, channel=1, num_classes=None):
    adam = optimizers.Adam()
    model = VGG16(weights='imagenet', include_top=True)
    model_res = Sequential()
    for layer in model.layers[:-1]: # just exclude last layer from copying
        model_res.add(layer)
    
    model_res.add(Dense(15,activation='softmax'))
    for layer in model_res.layers[:-1]:

        layer.trainable = False
    model_res.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    return model_res

In [15]:
model = vgg16_model(224, 224, 3, 15)

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [17]:
history = History()
model.fit(train_img,train_labels_dummy,batch_size=64,epochs=10,validation_split=0.15,shuffle=True,callbacks=[history])

Train on 1275 samples, validate on 225 samples
Epoch 1/10
1275/1275 [==============================] - 489s 384ms/step - loss: 1.0437 - accuracy: 0.6478 - val_loss: 0.4348 - val_accuracy: 0.8489
Epoch 2/10
1275/1275 [==============================] - 475s 373ms/step - loss: 0.2703 - accuracy: 0.9035 - val_loss: 0.3296 - val_accuracy: 0.8667
Epoch 3/10
1275/1275 [==============================] - 1559s 1s/step - loss: 0.1551 - accuracy: 0.9522 - val_loss: 0.3050 - val_accuracy: 0.8933
Epoch 4/10
1275/1275 [==============================] - 1030s 808ms/step - loss: 0.0906 - accuracy: 0.9796 - val_loss: 0.3289 - val_accuracy: 0.8711
Epoch 5/10
1275/1275 [==============================] - 609s 478ms/step - loss: 0.0577 - accuracy: 0.9914 - val_loss: 0.3234 - val_accuracy: 0.8889
Epoch 6/10
1275/1275 [==============================] - 465s 365ms/step - loss: 0.0413 - accuracy: 0.9961 - val_loss: 0.3201 - val_accuracy: 0.8711
Epoch 7/10
1275/1275 [==============================] - 517s 405ms

In [18]:
model.evaluate(test_img,test_labelsdummy)

2985/2985 [==============================] - 1869s 626ms/step


[0.37291230101331796, 0.8793969750404358]

In [26]:
train_labels_dummy.shape

(1500, 15)

In [27]:
# I predict classes of my test_data in order to build my confusion matrix 
y_pred = model.predict_classes(test_img)

KeyboardInterrupt: 

4

In [ ]:
utils.plot_confusion_matrix(cm,labels,"point_one","images_point_one/",normalize=True)